In [1]:
import pandas as pd

In [2]:
faults_df = pd.read_csv('../data/J1939Faults.csv')

C:\Users\jacob\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
pd.set_option('display.max_columns', 300)

In [4]:
faults_df.shape

(1187335, 20)

In [5]:
faults_df.dtypes

RecordID                   int64
ESS_Id                     int64
EventTimeStamp            object
eventDescription          object
actionDescription        float64
ecuSoftwareVersion        object
ecuSerialNumber           object
ecuModel                  object
ecuMake                   object
ecuSource                  int64
spn                        int64
fmi                        int64
active                      bool
activeTransitionCount      int64
faultValue               float64
EquipmentID               object
MCTNumber                  int64
Latitude                 float64
Longitude                float64
LocationTimeStamp         object
dtype: object

In [6]:
faults_df.ESS_Id.nunique()

1182354

In [7]:
faults_df['EventTimeStamp'] = faults_df['EventTimeStamp'].astype('datetime64[ns]')

In [8]:
faults_df['LocationTimeStamp'] = faults_df['LocationTimeStamp'].astype('datetime64[ns]')

In [30]:
derates = faults_df[((faults_df['spn'] == 1569) & (faults_df['fmi'] == 31))| (faults_df['spn'] == 5246)]

In [31]:
derates.shape

(12122, 20)

In [32]:
derates = derates[~((derates['Longitude'].astype('str').str.contains(r'-86.43\d+')) &
       (derates['Latitude'].astype('str').str.contains(r'36.06\d+'))
         )]

In [33]:
derates = derates[~((derates['Longitude'].astype('str').str.contains(r'-86.44\d+')) & (derates['Latitude'].astype('str').str.contains(r'35.58\d+')))]

In [34]:
derates = derates[~((derates['Longitude'].astype('str').str.contains(r'-83.17\d+')) & (derates['Latitude'].astype('str').str.contains(r'36.19\d+')))]

In [35]:
derates.shape

(10735, 20)

In [36]:
derates.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
40,41,990856,2015-02-21 12:06:22,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1721,105439606,39.051805,-84.560509,2015-02-21 12:06:57
249,250,993542,2015-02-21 15:01:49,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,False,5,NaN,1721,105439606,38.113240,-85.667361,2015-02-21 15:01:45
290,291,994045,2015-02-21 15:35:45,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,6,NaN,1721,105439606,37.735185,-85.808101,2015-02-21 15:36:21
306,307,994237,2015-02-21 15:51:58,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,False,6,NaN,1721,105439606,37.540370,-85.880972,2015-02-21 15:51:54
340,341,994680,2015-02-21 16:22:24,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,7,NaN,1721,105439606,37.166666,-85.964027,2015-02-21 16:23:00


In [37]:
faults_df['EquipmentID'].nunique()

1927

In [38]:
cumulative_active = pd.DataFrame(derates.groupby(['EquipmentID', 'EventTimeStamp','spn'])['active'].sum().groupby(level = 0).cumsum()).reset_index()

In [39]:
cumulative_active['diff'] = cumulative_active['active'].diff(periods=1)

In [40]:
cumulative_active['diff_time'] = cumulative_active['EventTimeStamp'].diff(periods = 1)

In [41]:
cumulative_active

,EquipmentID,EventTimeStamp,spn,active,diff,diff_time
0,301,2016-07-12 19:11:07,5246,1,NaN,NaT
1,302,2020-01-06 10:13:57,5246,1,0.0,1272 days 15:02:50
2,302,2020-01-13 12:15:04,5246,1,0.0,7 days 02:01:07
3,302,2020-01-13 13:18:31,5246,2,1.0,0 days 01:03:27
4,302,2020-01-22 21:15:03,5246,2,0.0,9 days 07:56:32
5,302,2020-02-14 11:21:54,5246,3,1.0,22 days 14:06:51
6,302,2020-02-17 17:08:39,5246,3,0.0,3 days 05:46:45
7,304,2016-02-15 10:59:28,5246,1,-2.0,-1464 days +17:50:49
8,304,2016-02-15 11:01:09,5246,1,0.0,0 days 00:01:41
9,305,2015-12-14 18:25:46,5246,1,0.0,-63 days +07:24:37


In [43]:
faults_df[(faults_df['EquipmentID'] == 302) & (faults_df['spn'] == 5246) & (faults_df['fmi'] == 31)]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp


In [22]:
onboard = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')

In [23]:
onboard

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1
...,...,...,...,...
12821621,12864020,EngineCoolantTemperature,181.4,1248457
12821622,12864021,ParkingBrake,False,1248457
12821623,12864022,SwitchedBatteryVoltage,14.1,1248457
12821624,12864023,DistanceLtd,28606.65625,1248457


In [24]:
derates['spn'].value_counts()

1569    9797
5246     938
Name: spn, dtype: int64

In [25]:
faults_df['ecuSerialNumber'].nunique()

1989

In [26]:
faults_df['EquipmentID'].nunique()

1927

In [27]:
derates.Latitude.to_list()

[39.051805,
 38.113240000000005,
 37.735185,
 37.54037,
 37.166666,
 37.079444,
 36.770324,
 36.753287,
 30.376435,
 39.638703,
 40.733009,
 25.878518,
 25.860277,
 25.861898,
 28.077361,
 38.288518,
 39.399583,
 39.399583,
 39.399629,
 39.399629,
 36.169027,
 37.596805,
 37.596805,
 37.596851,
 37.596851,
 37.596805,
 37.596805,
 38.966898,
 40.534259000000006,
 40.532962,
 41.225879,
 41.225648,
 41.033333,
 41.022824,
 39.942777,
 39.942777,
 39.943796,
 39.943796,
 39.943564,
 39.943888,
 39.943888,
 39.943888,
 39.945648,
 40.110787,
 40.109212,
 40.912916,
 41.125972,
 41.569907,
 39.970925,
 40.012453,
 40.905,
 40.811851,
 38.271111,
 37.772962,
 37.879722,
 37.572453,
 37.5725,
 37.543611,
 37.852129,
 37.862962,
 37.783009,
 37.772175,
 37.751435,
 37.982222,
 38.101435,
 39.13699,
 39.196203,
 39.518240000000006,
 39.639675,
 36.022268,
 36.182962,
 36.090416,
 34.60787,
 34.927361,
 34.918009000000005,
 34.715648,
 34.699721999999994,
 33.725648,
 33.670138,
 33.51375,
 34.